<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/ExemploRespondendoPerguntaTextoLongo_Llama2_Langchain_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Respondendo Perguntas sobre Textos Longos usando Llama v2.0, Longchain, Chroma e Transformers by HuggingFace

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

In [ ]:
!pip install pypdf==3.16.4

Dependência para manipular os embeddings pelo Langchain

In [ ]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Instala o Chroma para armazenar os embeddings

In [ ]:
!pip install chromadb==0.4.15

Instala langchain

In [ ]:
!pip install langchain==0.0.321

Dependências do xformers

In [ ]:
!pip install lmdb
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 torchtext==0.15.2+cpu torchdata==0.6.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.22.post4 requires torch==2.1.0, but you have torch 2.0.1+cu118 which is incompatible.


Permite maior velocidade e menor consumo de memória nos transformers.

In [ ]:
!pip install xformers==0.0.22.post4

  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.15.2+cpu requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have to

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes==0.41.1

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

In [ ]:
# accelerate: treino distribuído, mixed precision, consumer hardware
!pip install accelerate==0.23.0

Instala a interface pytorch para o BERT by Hugging Face.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

Instala o cliente do huggingface hub para realizar o login.

In [ ]:
!pip install huggingface_hub==0.18.0

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

ACCESS_TOKEN  = 'COLOQUE O SEU TOKEN DE ACESSO AQUI'

HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

#nome_modelo = "meta-llama/Llama-2-13b-hf"
# nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


## 2.4 - Carregando o Modelo LLM

Carregando o **modelo** da comunidade Huggingface.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import AutoModelForCausalLM
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True,   # Carrega de um repositório confiável
                                             load_in_8bit=True,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tempo de carregamento do modelo LLM:  0:01:15 (h:mm:ss)


In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



## 2.5 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    max_length=2048,
    #top_k=3,
    # num_return_sequences=1,
    # max_new_tokens = 10 # Limita a quantidade de tokens gerada
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.1})

In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': {'temperature': 0.1}, 'pipeline_kwargs': None}


## 2.6 - Carregando o ML BERT

Download do BERT via HuggingFace

Os modelos [BGE](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) no HuggingFace são os melhores modelos de embeddings de código aberto. O modelo BGE é criado pela Academia de Inteligência Artificial de Pequim(*Beijing Academy of Artificial Intelligence*-BAAI) . BAAI é uma organização privada sem fins lucrativos envolvida em pesquisa e desenvolvimento de IA.

In [ ]:
# Import das bibliotecas
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

#MODELO_BERT = "bert-large-cased"
#MODELO_BERT = "bert-base-cased"
MODELO_BERT = "neuralmind/bert-large-portuguese-cased"
#MODELO_BERT = "neuralmind/bert-base-portuguese-cased"

# Carrega os embeddings diretamente do HuggingFace
model_lm = HuggingFaceBgeEmbeddings(
    model_name=MODELO_BERT,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Tempo de carregamento do modelo LM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de carregamento do modelo LM:  0:00:12 (h:mm:ss)


In [ ]:
print(model_lm)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) model_name='neuralmind/bert-large-portuguese-cased' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: '


# 3 - Pergunta em texto longo

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## 3.1 - Carrega os documentos de PDF

### 3.1.1 - Download do PDF

Download do PDF do livro "As Vítimas Algozes".

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070

PDF: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977

In [ ]:
# Import das bibliotecas
# import subprocess

# # As Vítimas Algozes
# urlpdf = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# destino = 'arquivo1.pdf'

# # Executa o comando wget no prompt
# subprocess.call(["wget", urlpdf, "-O", destino])

### 3.1.2 - Carrega do PDF



In [ ]:
# # Import das bibliotecas
# from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# # Define o diretório
# diretorio = '/content'

# # Cria o carregar dos documentos do diretório
# # Pode ser usado o PyPDFLoader para um arquivo
# carregador = PyPDFDirectoryLoader(diretorio)

# # Carrega os documentos
# documentos = carregador.load()

## 3.2 - Carrega os documentos da WEB

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

### 3.2.1 Download da WEB

Download do PDF do livro "As Vítimas Algozes".

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070

HTML: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979

In [ ]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

url = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# Cria o carregador da página
carregador = WebBaseLoader(url)

# Carrega os documentos
documentos = carregador.load()

Exibe parte dos dados carregados

In [ ]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 0
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Medadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 1
Trecho página( 0 ) : 















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma ven

Medadados: {'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'title': 'As vítimas algozes - Joaquim Manuel de Macedo', 'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.'}


## 3.3 - Carrega e divide os documentos

Carrega o documentos e realiza o divisão do documento em pedaços(chunks) e faz a sobreposição(overlap) para garantir o contexto semântico entre os pedaços.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto no link: https://chunkerizer.streamlit.app/

In [ ]:
# Import das bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time

# Parâmetros
chunk_tamanho = 2048
chunk_sobreposicao = 750

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos documentos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos html em {tempo_final - tempo_inicio} segundos!")
print(f"Quantidade de chunks: {len(chunks)}")

Carregando e dividindo 1 documentos pdfs em 0.05613231658935547 segundos!


## 3.5 - Armazena os chunks

Utiliza o Chroma um banco de dados de embeddings de código aberto e leve.

https://www.trychroma.com/

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

In [ ]:
# Import das bibliotecas
from langchain.vectorstores import Chroma

# Armazena os chunks usando o Chroma
vectorstore = Chroma.from_documents(documents=chunks, embedding=model_lm, persist_directory="./bancodedados", )

## 3.5 - Função de envio de perguntas ao LLM

In [ ]:
# Import das bibliotecas
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch
import gc

def avaliarContexto(texto):

  # Cria o texto de prompt
  prompt_template = """Você é um assistente de IA útil e fornece a resposta para a pergunta com base no contexto fornecido.
Contexto: {context}
>>PERGUNTA<< {question}
>>RESPOSTA<<"""

  # Cria o prompt
  # As variáveis "context" e "question" não podem ser alteradas
  prompt = PromptTemplate(input_variables=["context", "question"],
                          template=prompt_template)

  # Instancia o chain
  chain_type_kwargs = {"prompt": prompt}
  chain = RetrievalQA.from_chain_type(
        llm=model_llm,
        chain_type="stuff",
        #retriever=vectorstore.as_retriever(), # Passa o contexto(livro carregado)
        retriever=vectorstore.as_retriever(search_kwargs={'k':1}), # Passa o contexto(livro carregado)
        chain_type_kwargs=chain_type_kwargs
    )

  # Executa o prompt no llm
  resultado = chain.run(texto)

  # Esvazia a memória
  # del chain
  # gc.collect()
  # torch.cuda.empty_cache()

  return resultado

## 3.6 - Algumas perguntas usando o contexto

Guarda o tempo de início de realização das perguntas

In [ ]:
# Importando as bibliotecas do Modelo
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

In [ ]:
texto = "Qual o período que ocorre a história do texto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 O período que ocorre a história do texto é o Segundo Império, pois é mencionado o Rio de Janeiro e a cidade de São Paul
o, que eram as capitais do Brasil na época. Além disso, é mencionado o império, o que indica que o texto ocorre durante 
o Segundo Império, que durou de 1852 a 1870.


In [ ]:
texto = "Considerando o contexto em qual período que ocorre os fatos?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 O contexto é do século XIX, no Brasil, no início do século, cerca de 1820.


In [ ]:
texto = "Quem são os principais personagens do contexto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os principais personagens do contexto são:

1. Deus: é o criador do universo e dos seres humanos, e é considerado o sen
hor e o dono de todos.
2. Homens: são os seres humanos, que são considerados como escravos de Deus e de seus opressores.

3. Escravos: são os seres humanos que são considerados como miseráveis e degradados pela condição violentada em que viv
em.
4. Opressores: são os seres humanos que opressam e exploram os escravos, e são considerados como responsáveis pela e
scravidão.
5. Esposas e filhas: são as mulheres que são relacionadas com os homens e os escravos, e são consideradas com
o sendo vulneráveis ao contacto e à influência dos escravos.

Esses são os principais personagens do contexto, e eles sã
o relacionados entre si de maneira complexa e interdependente.


In [ ]:
texto = "Considerando o contexto quem são os principais personagens da história?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os principais personagens da história são os escravos, que são descritos como "miseráveis degradados pela condição viol
entada", "engolfados nos vícios mais torpes", "materializados", "corruptos", "apodrecidos na escravidão" e "pestíferos p
elo viver no pantanal da peste". Eles são vistos como sendo "com vossas esposas", "com vossas filhas", e como tendo um i
mpacto negativo na sociedade escravagista.


In [ ]:
texto = "Em que pais e cidade ocorreram os fatos?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Os fatos ocorreram no Brasil, no século XIX, durante a época da escravidão. A obra "Bani a Escravidão!" é de José de Al
encar, um escritor brasileiro do século XIX, e foi publicada em 1870. A obra é uma crítica à escravidão e aos opressores
 que a praticavam no Brasil.


In [ ]:
texto = "Em que pais e cidade ocorre os fatos do contexto?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 O contexto ocorre em uma cidade do Brasil, não especificada, na época do Império Braziliense.


In [ ]:
texto = "Quais palavras possuem relação com escravidão?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 As palavras que possuem relação com escravidão são:

* Deus
* senhores
* donos
* escravos
* peste
* tão vis
* tão perig
osos
* desmoralizando
* envenenando
* desonrando
* empestando
* assassinando
* Bani a escravidão

Essas palavras são usa
das no contexto da obra "O Escravo Antes do Café" de José de Alencar, onde o autor critica a escravidão e a opressão que
 os escravos sofrem nas mãos dos senhores.


In [ ]:
texto = "Considerando o contexto que palavras possuem relação com escravidão?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 As palavras que possuem relação com a escravidão são:

1. Senhores: Refere-se aos proprietários de escravos, que tinham
 o direito de possuir e explorar trabalhadores escravos.
2. Escravos: Refere-se aos trabalhadores que eram propriedade d
e outros e eram obrigados a realizar trabalhos forçados.
3. Donos: Refere-se aos proprietários de escravos, que tinham o
 direito de possuir e explorar trabalhadores escravos.
4. Árbitros: Refere-se aos indivíduos que tinham o poder de julga
r e resolver conflitos entre os escravos e seus proprietários.
5. Peste: Refere-se à epidemia de doenças que afetava os 
escravos e seus proprietários, tornando-os mais vulneráveis à escravidão.
6. Tão vis, tão perigosos: Refere-se à ideia d
e que os escravos eram vistos como uma ameaça para a sociedade, e que sua presença era perigosa para os proprietários.
7
. Empestando, assassinando: Refere-se à ideia de que a escravidão era uma doença que afetava a sociedade como um todo, e
 que era capaz de matar a socied

In [ ]:
texto = "Considerando o contexto que palavras possuem relação semântica com escravidão?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 As palavras que possuem relação semântica com escravidão são:

1. Opressores: Refere-se a um indivíduo ou grupo que exe
rce o poder de maneira cruenta ou oppressiva sobre outros.
2. Desonrando: Significa desacreditar ou desvirginar algo ou 
alguém, relacionando-se à ideia de que a escravidão é uma forma de desonorar os escravos e sua condição humana.
3. Empes
tando: Refere-se a uma substância ou ação que cause um cheiro desagradável ou nocivo, relacionando-se à ideia de que a e
scravidão é uma forma de "empestar" os escravos e a sociedade como um todo.
4. Assassinando: Significa matar ou causar a
 morte de alguém, relacionando-se à ideia de que a escravidão é uma forma de assassinato cultural e social dos escravos.

5. Desmoralizando: Refere-se a uma ação ou situação que cause a perda da moral ou da integridade de alguém, relacionand
o-se à ideia de que a escravidão é uma forma de desmoralizar os escravos e sua condição humana.
6. Envenenando: Signific
a causar danos ou problemas a al

In [ ]:
texto = "Considerando o contexto que palavras estão relacionadas a escravidão mas deixa isto implicito?"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

 Sim, é possível que as palavras relacionadas a escravidão estejam implicitamente presentes na obra, mesmo que não estej
am explicitamente mencionadas. A escravidão era uma realidade da sociedade brasileira no tempo da obra e pode ter sido r
eferenciada de forma implícita através de imagens, metáforas ou outros elementos literários. Por exemplo, a ideia de "mi
seráveis degradados pela condição violentada" pode ser uma referência à escravidão, mesmo que não esteja explicitamente 
mencionada. Além disso, a obra pode conter críticas à escravidão de forma implícita através de personagens ou situações 
que representam a opposição à escravidão.


Texto do trabalho do **Leandro**

In [ ]:
texto = "Considerando a palavra \"negro\" com alvo, identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Ao considerar a palavra "negro" com alvo, outras palavras semanticamente similares que podem ser identificadas no conte
xto são:

* Preto
* Escravo
* Africano
* Mulato
* Moreno

Essas palavras compartilham a mesma conotação negativa e de op
ressão que a palavra "negro" com alvo, e são frequentemente usadas de forma intercambiável para referir pessoas de cor e
scura. No entanto, é importante notar que essas palavras não são sinônimas exatas, e cada uma tem um contexto e uma hist
ória específicos.


In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ lista as palavras possuem ligação com a palavra \'negro\'"

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Na sentença 'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da v
ingança.' as palavras 'escravo', 'negro' e'vergonha' estão relacionadas.


In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ lista as palavras possuem ligação com a palavra alvo \'negro\' considerando o contexto."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Na sentença 'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da v
ingança.' as palavras 'negro' e 'escravo' estão relacionadas, pois o escravo é descrito como 'negro'. No contexto da his
tória, o termo 'negro' é usado para descrever os escravos, que são pessoas de cor preta. Portanto, a resposta à pergunta
 é que as palavras 'negro' e 'escravo' estão relacionadas no sentido de que o escravo é descrito como um 'negro'.


In [ ]:
texto = "Usando leitura distante realize a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A sentença 'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vin
gança.' contém as seguintes palavras que possuem ligação com a palavra alvo 'negro':

1. falta
2. escravidão
3. negro
4.
 rancor
5. vingança

Essas palavras possuem uma conexão semântica com a palavra 'negro', pois elas estão relacionadas ao
 tema da escravidão e ao status social dos negros na época.


In [ ]:
texto = "Considerando o contexto use leitura distante para realizar a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Ao considerar a sentença 'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o
 desejo da vingança.' no contexto da história, as palavras que possuem ligação com a palavra alvo 'negro' são:

1. escra
vidão
2. escravo
3. negro
4. pundonor
5. vergonha
6. rancor
7. vingança

Essas palavras possuem uma conexão semântica co
m a palavra alvo 'negro', pois são usadas para descrever a condição social e a situação de opressão dos escravos, que er
am majoritariamente negros. Ao considerar a sentença no contexto da história, é importante lembrar que a escravidão era 
uma prática comum no Brasil colonial, e que os escravos eram majoritariamente negros.


In [ ]:
texto = "Usando leitura distante realize a analise da sentença: \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ e liste as palavras possuem relação semântica com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

print_linhas_menores(resultado)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras que possuem relação semântica com a palavra alvo 'negro' na sentença analisada são:

* rancor
* desejo
* vi
ngança
* escravidão
* pundonor
* vergonha

Essas palavras compartilham a mesma semântica de 'negro' como 'escravo','mise
rável', 'degradado', 'violentado', 'pestífero', 'torpe', 'perigoso', 'obrigado', 'fatal', 'envenenando', 'desonrando', '
empestando', 'assassinando'.


Tempo final de execução das perguntas

In [ ]:
print("Tempo de execução das perguntas:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de execução das perguntas:  0:12:03 (h:mm:ss)
